<a href="https://colab.research.google.com/github/fzanart/Socialz/blob/main/mu_lambda_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scipy --upgrade

In [9]:
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import qmc

In [10]:
data = {}


def generate_random(number):
  values = []
  for i in range(10):
      values.extend([random.uniform(0, 1)])
  return values

data['Betweeness'] = generate_random(10)
data['Average_activity'] = generate_random(10)
data['Abnormal_ratio'] = generate_random(10)

df = pd.DataFrame(data)
df

,Betweeness,Average_activity,Abnormal_ratio
0,0.614062,0.641559,0.263238
1,0.654887,0.162797,0.175473
2,0.411365,0.218218,0.697543
3,0.290461,0.779146,0.744818
4,0.849185,0.109536,0.827267
5,0.904911,0.473598,0.789840
6,0.446492,0.547697,0.734179
7,0.175780,0.203725,0.474223
8,0.848459,0.633665,0.467135
9,0.931052,0.880664,0.963333


mu: The number of parents selected each iteration.

lambda: Size of the population.

lambda / mu: Number of children generated from each selected parent.

In [92]:
qmc.discrepancy(df.to_numpy(), method='L2-star',workers=-1)

0.06368088180876683

In [14]:
def objective(initial_population, candidate):
  space = initial_population.to_numpy()
  space = np.vstack((space, candidate))
  return qmc.discrepancy(space, method='L2-star',workers=-1)

In [89]:
# check if a point is within the bounds of the search
def in_bounds(point, bounds):
	# enumerate all dimensions of the point
	for d in range(len(bounds)):
		# check if out of bounds for this dimension
		if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
			return False
	return True

In [90]:
# evolution strategy (mu, lambda) algorithm
def es_comma(objective, bounds, n_iter, step_size, mu, lam):
	best, best_eval = None, 1e+10
	# calculate the number of children per parent
	n_children = int(lam / mu)
	# initial population
	population = list()
	for _ in range(lam):
		candidate = None
		while candidate is None or not in_bounds(candidate, bounds):
			candidate = bounds[:, 0] + np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
		population.append(candidate)
	# perform the search
	for epoch in range(n_iter):
		# evaluate fitness for the population
		scores = [objective(df, c) for c in population]
		# rank scores in ascending order
		ranks = np.argsort(np.argsort(scores))
		# select the indexes for the top mu ranked solutions
		selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]
		# create children from parents
		children = list()
		for i in selected:
			# check if this parent is the best solution ever seen
			if scores[i] < best_eval:
				best, best_eval = population[i], scores[i]
				print('%d, Best: (%s) = %.5f' % (epoch, best, best_eval))
			# create children for parent
			for _ in range(n_children):
				child = None
				while child is None or not in_bounds(child, bounds):
					child = population[i] + np.random.randn(len(bounds)) * step_size
				children.append(child)
		# replace population with children
		population = children
	return [best, best_eval]

In [91]:
# define range for input
bounds = np.asarray([[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]])
# define the total iterations
n_iter = 5000
# define the maximum step size
step_size = 0.15
# number of parents selected
mu = 20
# the number of children generated by parents
lam = 100
# perform the evolution strategy (mu, lambda) search
best, score = es_comma(objective, bounds, n_iter, step_size, mu, lam)
print('(%s) = %f' % (best, score))

0, Best: f([0.55494024 0.3177519  0.18322595]) = 0.06221
0, Best: f([0.28687834 0.68565626 0.49322849]) = 0.05981
0, Best: f([0.43143179 0.89198558 0.1054242 ]) = 0.05942
0, Best: f([0.16033003 0.57356178 0.42532656]) = 0.05721
0, Best: f([0.37080323 0.43087518 0.21886505]) = 0.05408
0, Best: f([0.05058689 0.71347696 0.00414167]) = 0.05355
1, Best: f([0.10572367 0.60085753 0.17859806]) = 0.05239
1, Best: f([0.08687098 0.57745599 0.21997134]) = 0.05220
2, Best: f([0.24415256 0.58145931 0.06847236]) = 0.05183
2, Best: f([0.12677881 0.47718048 0.22297827]) = 0.05161
2, Best: f([0.12607332 0.57495417 0.08845359]) = 0.05138
2, Best: f([0.21001087 0.46489243 0.12150665]) = 0.05104
3, Best: f([0.12052145 0.49032309 0.1120397 ]) = 0.05071
52, Best: f([0.10786047 0.5084625  0.10378636]) = 0.05070
80, Best: f([0.11776317 0.49959932 0.11525603]) = 0.05069
311, Best: f([0.11586735 0.49416038 0.10857676]) = 0.05068
1332, Best: f([0.10625531 0.49730091 0.11310836]) = 0.05067
1434, Best: f([0.1082216